In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [36]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
lemmatizer = WordNetLemmatizer()
from itertools import combinations
from nltk.corpus import wordnet 
import nltk
nltk.download('wordnet')
from gensim.parsing.preprocessing import remove_stopwords
from numpy import dot,mean
from numpy.linalg import norm
from sklearn.model_selection import train_test_split, cross_val_score
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")
import joblib
import math

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


reading the dataset

In [4]:
dataset=pd.read_csv("/content/gdrive/MyDrive/dis_sym_dataset_comb.csv",encoding='utf-8')
dataset2=pd.read_csv("/content/gdrive/MyDrive/temp.csv")

In [5]:
dataset2

,label_dis,abdominal cramp,abdominal distention,abnormal behavior,abnormal bleeding,abnormal sensation,abnormally frequent,abscess,aching,acne,...,wet,wheezing,white patch vaginal discharge,widespread pain,withdrawal occurring stopping,worrying,yellow skin,yellowish coloration skin white eye,yellowish skin,yellowish skin crust
0,Abscess,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acquired Capillary Haemangioma of Eyelid,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acquired Immuno Deficiency Syndrome,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acute encephalitis syndrome,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Adult Inclusion Conjunctivitis,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Yaws,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
257,Yellow Fever,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
258,Zika virus disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259,lactose intolerance,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
data_symptom=list(dataset.columns)
data_symptom.remove('label_dis')
data_symptom

['abdominal cramp',
 'abdominal distention',
 'abnormal behavior',
 'abnormal bleeding',
 'abnormal sensation',
 'abnormally frequent',
 'abscess',
 'aching',
 'acne',
 'acquiring drinking alcohol taking lot time',
 'affected part turning white',
 'anemia',
 'anxiety',
 'arm',
 'attack pain',
 'back',
 'bacterial infection',
 'bad breath',
 'bad smelling thin vaginal discharge',
 'bad smelling vaginal discharge',
 'barky cough',
 'belching',
 'better sitting worse lying',
 'birth baby younger week gestational age',
 'bleeding gum',
 'bleeding skin',
 'blindness',
 'blindness one eye',
 'blister sunlight',
 'bloating',
 'blood stool',
 'blood urine',
 'bloody diarrhea',
 'blue',
 'bluish skin coloration',
 'blurred vision',
 'blurry vision',
 'body tremor',
 'bone pain',
 'bowed leg',
 'breakdown skeletal muscle',
 'breathing problem',
 'bruising',
 'burning',
 'burning redness eye',
 'burning stabbing pain',
 'burning urination',
 'certain thought repeatedly',
 'change bowel movement',

In [7]:
#function to get synonym of a given term
def syno(token):
  synonym = []
  website = 'https://www.synonym.com/synonyms/';
  myLinkFinder = website + token
  synoPage = requests.get(myLinkFinder,verify=False)
  currContent = BeautifulSoup(synoPage.content, 'html5lib')
  html_Div = currContent.find('div', class_='section-list-wrapper show')
  row = html_Div.find_all('li')
  for i in row:
    synonym.append(i.get_text())
  for i in wordnet.synsets(token):
    synonym+=i.lemma_names()
  return set(synonym)

#function to lemmetize the word
def lemme(txt):
  txt=nltk.word_tokenize(txt)
  temp=[]
  for i in txt:
    temp.append(WordNetLemmatizer().lemmatize(i))
  return temp

#function to calculate jaccard coefficient of 2 set
def getJaccard(setA,setB):
  #print(setA,setB)
  return(len(setA.intersection(setB))/len(setA))



#fucntion to calculate cosine similarity between 2 lists
def cos_sim(listA, listB):
  cos_sim = dot(listA, listB) / (norm(listA) * norm(listB))
  return cos_sim

#function to process the input symptoms
def refine_symptoms(input_symptoms):
  input_symptoms=input_symptoms.lower()
  input_symptoms=input_symptoms.split(",")
  refined_symptoms=[]
  for i in input_symptoms:
    i=i.replace("-"," ")
    i=i.replace("'","")
    refined_symptoms.append(i)
  print(refined_symptoms)
  return refined_symptoms

#function to add synonyms of the symptoms in the list and extend the symptom list
def extend_symptoms(refined_symptoms):
  for sym in refined_symptoms:
  #remove stopwords if any before taking its synonym
    sym=remove_stopwords(sym)
    sym = sym.split()
    set_sym = set()
    for comb in range(1, len(sym)+1):
        for sub_set in combinations(sym, comb):
            sub_set=' '.join(sub_set)
            sub_set = syno(sub_set) 
            set_sym.update(sub_set)
    extended_symptom.append(' '.join(set_sym))
  print(extended_symptom)
  return extended_symptom

#selecting relevent symptoms from the dataset
def relevent_symptom(data_symptom,extended_symptom):
  our_symptom=[]
  for sym in data_symptom:
    data_sym_tokens=sym.split()
    for user_sym in extended_symptom:
      if getJaccard(set(data_sym_tokens),set(user_sym.split()))>0.5:
        our_symptom.append(sym)
  print(our_symptom)
  return our_symptom

#confirming from user , the symptoms present in dataset
def confirm_relevent_symptom(our_symptom):
  poss_disease=set()
  input_list=input("\n just confirm the relevent symptoms by entering their indices separated by comma\n").split(",")
  for i in input_list:
    new_symptom.append(our_symptom[int(i)-1])
    poss_disease.update(list(dataset2[dataset2[our_symptom[int(i)-1]]==1]['label_dis']))
  return new_symptom,poss_disease

#selecting co-occuring symptoms from the dataset
def getCoOccuringSymptoms(new_symptom):
  for disease in poss_disease:
    temp_values = dataset2.loc[dataset2['label_dis'] == disease].values
    temp_list=list(temp_values[0])
    del temp_list[0]
    index=0
    for i in temp_list:
      if i==1 and data_symptom[index] not in new_symptom:
        co_occur_symptom.append(data_symptom[index])  
      index+=1
  return co_occur_symptom

#gives most probable diseases
def giveBestDiseases(predicted_list):
  bestK = predicted_list[0].argsort()
  bestK=bestK[-7:]
  bestK=bestK[::-1]
  return bestK

#get probabilities of the selected diseases
def get_probability(disease,new_symptom,bestK):
  for i in bestK:
    value_list=list(dataset2.loc[dataset2["label_dis"]==disease[i]].values)
    value_list=list(value_list[0])
    del value_list[0]
    ind=0
    for j in value_list:
      if j==1:
        dis_set.add(data_symptom[ind])
      ind+=1
    pod=(len(dis_set.intersection(set(new_symptom)))+1)/(len(new_symptom)+1)
    disease_prob[i]=pod*100
  return disease_prob

taking input from the user

In [8]:
input_symptoms=str(input("enter symptoms separated by comma"))
refined_symptoms=refine_symptoms(input_symptoms)

enter symptoms separated by commacough,vomiting,fever
['cough', 'vomiting', 'fever']


extending the input symptoms so that we get more words....which will help in matching and prediction

In [9]:
extended_symptom = []
extended_symptom = extend_symptoms(refined_symptoms)

['hawk expectorate cough out cough clear the throat coughing hack spit up whoop spit out', 'retch sick rumination reflex response puking spue reflex vomit_up projection reflex action cat physiological reaction throw_up barf inborn reflex innate reflex disgorge upchuck hematemesis chuck purge unconditioned reflex regorge hyperemesis be_sick emesis spew vomit disgorgement haematemesis regurgitation vomiting cast regurgitate ejection forcing out expulsion puke honk', 'pyrexia feverishness symptom hyperpyrexia febricity febrility fever']


selecting relevant symptoms from the dataset

In [10]:
our_symptom=[]
our_symptom= relevent_symptom(data_symptom,extended_symptom)

['coughing', 'fever', 'vomiting']


asking to confirm symptoms of our dataset matching with user input

In [25]:
print("This are matching system with the given symptoms")
for i in range(0,len(our_symptom)):
  print(i+1," ",our_symptom[i])

new_symptom=[]
new_symptom,poss_disease=confirm_relevent_symptom(our_symptom)

This are matching system with the given symptoms
1   coughing
2   fever
3   vomiting

 just confirm the relevent symptoms by entering their indices separated by comma
1,2,3


selecting co-occuring symptoms

In [26]:
co_occur_symptom=[]
co_occur_symptom=getCoOccuringSymptoms(new_symptom) 

sorting co-occuring symptom

In [13]:
from collections import Counter
import operator
dict_symp = dict(Counter(co_occur_symptom))
dict_symp_sorted = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)

adding co-occuring symptoms in the final list

In [14]:
new_symptom.append(dict_symp_sorted[0][0])
new_symptom.append(dict_symp_sorted[1][0])

creating sample vector for our final symptoms

In [56]:
my_sample=[]
for x in range(0,len(data_symptom)):
  my_sample.append(0)
for val in new_symptom:
  my_sample[data_symptom.index(val)]=1

Logistic Regression 

In [16]:
from sklearn.linear_model import LogisticRegression
x_of_data = dataset2.iloc[:, 1:]
y_of_data = dataset2.iloc[:, 0:1]

log_reg = LogisticRegression()
log_reg = log_reg.fit(x_of_data, y_of_data)

predicted_list = log_reg.predict_proba([my_sample])
#joblib.dump(log_reg,"log_reg.pkl")

KNN model 

In [17]:
from sklearn.neighbors import KNeighborsClassifier
x_of_data = dataset2.iloc[:, 1:]
y_of_data = dataset2.iloc[:, 0:1]
naiveBayes = KNeighborsClassifier(n_neighbors=7)
naiveBayes.fit(x_of_data, y_of_data)
predicted_list = naiveBayes.predict_proba([my_sample])
joblib.dump(naiveBayes,"naiveBayes.pkl")

['naiveBayes.pkl']

getting best k probilities from the set of probabilities 

In [18]:
bestK=giveBestDiseases(predicted_list)


calculating probabilities

In [19]:
disease = list(y_of_data['label_dis'])
disease.sort()
disease_prob={}
dis_set=set()
disease_prob=get_probability(disease,new_symptom,bestK)

printing all the probable diseases

In [20]:
dict_final_sorted = sorted(disease_prob.items(), key=operator.itemgetter(1),reverse=True)
ind=1
old=0
for i in dict_final_sorted:
  if(old==disease_prob[i[0]]):
    print(ind,disease[i[0]],disease_prob[i[0]],"%")
  else:
    print(ind,disease[i[0]],disease_prob[i[0]],"%")
  ind+=1
  old=disease_prob[i[0]]

1 Paratyphoid fever 83.33333333333334 %
2 Rocky Mountain spotted fever 83.33333333333334 %
3 Rift Valley fever 83.33333333333334 %
4 Leptospirosis 83.33333333333334 %
5 Crimean Congo haemorrhagic fever (CCHF) 83.33333333333334 %
6 Bubonic plague 83.33333333333334 %
7 Malaria 66.66666666666666 %


TF-IDF + cosine similarity

here in our case only binary TF can work because 
if we consider the disease as a document and symptom as a token of the document then in each document the token can be present single time only. 

calculating idf for each symptom

In [37]:
idf_dict={}
for i in data_symptom:
  temp_list=list(dataset2[i])
  idf_dict[i]=math.log(temp_list.count(1)/(len(dataset2)+1),2)

making a new dataset to store tfidf value

In [39]:
dataset2_tfidf = dataset2.copy()

filling the tfidf value to dataframe

In [47]:
for row in range(0,len(dataset2)):
  for col in data_symptom:
    dataset2_tfidf.loc[row,col]=dataset2.loc[row,col]*idf_dict[col]

creating tfidf vector for input symptoms

In [55]:
my_sample_vector=[]
for x in range(0,len(data_symptom)):
  my_sample_vector.append(0)
for val in new_symptom:
  my_sample_vector[data_symptom.index(val)]=1*idf_dict[val]

predicting the disease using cosine similarity score between the query and document vector

In [59]:
dic_TF_IDF={}
for i in range(0,len(dataset2_tfidf)):
  temp=list(dataset2_tfidf.iloc[i])
  first_element=temp[0]
  del temp[0]
  dic_TF_IDF[first_element]=cos_sim(temp,my_sample_vector)
dict_final_sorted = sorted(dic_TF_IDF.items(), key=operator.itemgetter(1),reverse=True)
j=0
for i in dict_final_sorted:
  if j>6:
    break
  print(i[0],i[1]*100,"%")
  j+=1

Brucellosis 83.26183751146185 %
Malaria 46.259559046662076 %
Influenza 44.15470207353713 %
Asthma 39.34440904161031 %
Gastroenteritis 33.9275484281948 %
Bubonic plague 30.930109395354826 %
Trichinosis 28.553874674079303 %
